In [ ]:
# Preperation for obs and action plotting
import pandas as pd
from src.main.rl.evaluation.plots.phase3_plots import prepare_one_combination_actions_and_obs_for_analysis
from src.main.rl.utils.constants import color_mapping
from src.main.rl.utils.constants import (
    scaling_factors_scenario_1,
    scaling_factors_scenario_2,
    scaling_factors_scenario_3,
    action_dimensions_german,
)
import matplotlib.pyplot as plt

paths = [
    "src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption4Wrapper_None_RewardOption2Wrapper_TD3_training_04_06",
    "src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_SAC_training_04_06",
    "src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06",
    "src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption4Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_PPO_training_04_06",
    "src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06",
    "src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption3Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_PPO_training_04_06",
]
list_actions_prepared = []
list_obs_prepared = []
list_list_of_all_actions_taken = []
list_list_of_all_obs_taken = []
obs_dimensions = [6, 11, 11, 6, 11, 7]
scaling_functions = [
    lambda x: int(round((x + 1) * (scaling_factors[list_idx][idx] / 2))),
    lambda x: int(round((x + 1) * (scaling_factors[list_idx][idx] / 2))),
    lambda x: -scaling_factors_scenario_2[idx] if x == 0 else scaling_factors_scenario_2[idx],
    lambda x: -scaling_factors_scenario_2[idx] if x == 0 else scaling_factors_scenario_2[idx],
    lambda x: scaling_factors_scenario_3[idx][x],
    lambda x: scaling_factors_scenario_3[idx][x],
]
scaling_factors = [
    scaling_factors_scenario_1,
    scaling_factors_scenario_1,
    scaling_factors_scenario_2,
    scaling_factors_scenario_2,
    scaling_factors_scenario_3,
    scaling_factors_scenario_3,
]
for idx, path in enumerate(paths):
    (
        actions_prepared,
        obs_prepared,
        list_of_all_actions_taken,
        list_of_all_obs_taken,
    ) = prepare_one_combination_actions_and_obs_for_analysis(path, obs_dimensions=obs_dimensions[idx])
    list_actions_prepared.append(actions_prepared)
    list_obs_prepared.append(obs_prepared)
    list_list_of_all_actions_taken.append(list_of_all_actions_taken)
    list_list_of_all_obs_taken.append(list_of_all_obs_taken)

In [ ]:
###### Create all actions taken mean (min, max) for all 6 combis #######
for list_idx, actions_prepared in enumerate(list_actions_prepared):
    for idx, item in enumerate(actions_prepared):
        item = item.applymap(scaling_functions[list_idx])
        actions_prepared[idx] = item.agg(["mean", "min", "max", "std"], axis=1)
        actions_prepared[idx]["minus"] = actions_prepared[idx]["mean"] - actions_prepared[idx]["std"]
        actions_prepared[idx]["plus"] = actions_prepared[idx]["mean"] + actions_prepared[idx]["std"]
    fig, ax = plt.subplots(1, 5, constrained_layout=True)
    fig.set_figheight(3)
    fig.set_figwidth(20)
    [ax[idx].plot(item["mean"], color=color_mapping["standard"]) for idx, item in enumerate(actions_prepared)]
    [
        ax[idx].fill_between(
            [i for i in range(251)], item["min"], item["max"], color=color_mapping["standard"], alpha=0.3
        )
        for idx, item in enumerate(actions_prepared)
    ]
    [ax[idx].set_xlabel("Zeitschritte") for idx in range(5)]
    [ax[idx].set_ylabel("Value") for idx in range(5)]
    [ax[idx].set_title(action_dimensions_german[idx]) for idx in range(5)]
    plt.show()

In [ ]:
###### Create all obs taken mean (min, max) for all 6 combis #######
from src.main.rl.utils.constants import (
    scaling_factors_scenario_1,
    scaling_factors_scenario_2,
    action_dimensions_german,
    obs_dimensions_german,
    obs_scaling_factors,
    obs_dimensions,
    scaling_factors_scenario_3,
)

obs_dimensions = [6, 11, 11, 6, 11, 7]

for list_idx, obs_prepared in enumerate(list_obs_prepared):
    obs_space_size = obs_dimensions[list_idx]
    current_obs_scaling_factors = obs_scaling_factors[obs_space_size]
    agg_result_list_obs = [pd.DataFrame() for _ in range(obs_space_size)]
    # y_axis_scale=[[0,1000],[0,4000],[0,550],[0,8000],[0,180],[0,32]]
    fig, ax = plt.subplots(1, int(obs_space_size), constrained_layout=True)
    fig.set_figheight(3)
    fig.set_figwidth(30)
    for idx, item in enumerate(obs_prepared):
        item = item.fillna(0)
        item = item.applymap(lambda x: int(round((x + 1) * (current_obs_scaling_factors[idx] / 2))))
        agg_result_list_obs[idx] = item.agg(["mean", "min", "max", "std"], axis=1)
        agg_result_list_obs[idx]["minus"] = agg_result_list_obs[idx]["mean"] - agg_result_list_obs[idx]["std"]
        agg_result_list_obs[idx]["plus"] = agg_result_list_obs[idx]["mean"] + agg_result_list_obs[idx]["std"]
        # [ax[idx].plot(item[column],color=color_mapping["standard"]) for idx, item in enumerate(no_agg_result_list_obs)]

    [ax[idx].plot(item["mean"], color=color_mapping["standard"]) for idx, item in enumerate(agg_result_list_obs)]
    [
        ax[idx].fill_between(
            [i for i in range(251)], item["min"], item["max"], color=color_mapping["standard"], alpha=0.3
        )
        for idx, item in enumerate(agg_result_list_obs)
    ]
    [ax[idx].set_xlabel("Zeitschritte") for idx in range(obs_space_size)]
    [ax[idx].set_ylabel("Value") for idx in range(obs_space_size)]
    [ax[idx].set_title(obs_dimensions_german[obs_space_size][idx]) for idx in range(obs_space_size)]
    # [ax[idx].axhline(red_critical_line_1[idx],color=color_mapping["red"],ls='--') for idx in range(obs_space_size)]
    # [ax[idx].axhline(red_critical_line_2[idx],color=color_mapping["red"],ls='--') for idx in range(obs_space_size)]
    # [ax[idx].axhline(orange_critical_line_1[idx],color=color_mapping["orange"],ls='--') for idx in range(obs_space_size)]
    # [ax[idx].axhline(orange_critical_line_2[idx],color=color_mapping["orange"],ls='--') for idx in range(obs_space_size)]
    # [ax[idx].axhline(dead_line_1[idx],color="black",ls='--') for idx in range(obs_space_size)]
    # [ax[idx].axhline(dead_line_2[idx],color="black",ls='--') for idx in range(obs_space_size)]
    # [ax[idx].set_ylim(y_axis_scale[idx]) for idx in range(obs_space_size)]

    # fig.savefig(
#    f"src/main/rl/evaluation/plot_results/phase3_obs_plot_with_min_max_scen1_wo_all_models.png",
#    format="png",
#    dpi=300,
# )
# plt.show()

In [ ]:
# Create Stripe Plot
import seaborn
import matplotlib.pyplot as plt

paths = [
    "src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption4Wrapper_None_RewardOption2Wrapper_TD3_training_04_06",
    "src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_SAC_training_04_06",
    "src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06",
    "src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption4Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_PPO_training_04_06",
    "src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06",
    "src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_A2C_training_04_06",
]
path_to_save = "src/main/rl/evaluation/output/phase3_evaluation_results_new3.csv"
pd.options.display.max_colwidth = 500
df = pd.DataFrame()
try:
    df = pd.read_csv(path_to_save)
except:
    pass

if df.empty:
    df = create_evaluation_df_phase3(path_to_save, paths)
df
seaborn.set(style="white")
# loading data-set
# TODO add ticks
ax = seaborn.stripplot(
    x=df["combination"],
    y=df["cum_reward"],
    jitter=0.1,
    color=color_mapping["standard"],
    dodge=True,
    size=6,
)
ax.set(xticklabels=["1", "2", "3", "4", "5", "6"])
ax.set_xlabel("Kombination")
ax.set_ylabel("Return")
ax.margins(x=0.1)
ax.set(yticks=[160, 180, 200, 220, 240])
fig = ax.get_figure()
plt.tight_layout = True
plt.show()
fig.savefig(
    f"src/main/rl/evaluation/plot_results/phase3_rewards_per_combi.png",
    format="png",
    dpi=300,
)

In [ ]:
# Statistics per combination
from scipy.stats import iqr

df_statistics_per_combination = (
    df.drop(columns=["full_path"])
    .groupby(["combination", "scenario", "alg", "action_wrapper", "obs_wrapper", "automation_wrapper"], dropna=False)
    .agg(["mean", "max", "min", "std", iqr])
)
# Necessary to set index as alphabetical is confusing for the thesis
df_statistics_per_combination["index"] = [0, 1, 3, 2, 5, 4]
df_statistics_per_combination = (
    df_statistics_per_combination.reset_index().set_index(["index", "combination"]).sort_index()
)
df_statistics_per_combination.columns = ["_".join(a) for a in df_statistics_per_combination.columns.to_flat_index()]
save_df_per_combination = df_statistics_per_combination.copy()
[col for col in df_statistics_per_combination.columns if "time" in col]
df_statistics_per_combination[["total_timesteps_mean"]]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
from src.main.rl.evaluation.plots.phase3_plots import prepare_one_combination_actions_and_obs_for_analysis

path = "src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_A2C_training_04_06"

(
    actions_prepared,
    obs_prepared,
    list_of_all_actions_taken,
    list_of_all_obs_taken,
) = prepare_one_combination_actions_and_obs_for_analysis(path, obs_dimensions=11)

# Actions Analysis Combi 2

scaling_factors_scenario_1 = [100, 2000, 1, 1, 2000]

for idx, item in enumerate(actions_prepared):
    item = item.fillna(4)
    item = item.applymap(lambda x: scaling_factors_scenario_3[idx][x])
    actions_prepared[idx] = item.agg(["mean", "min", "max", "std"], axis=1)
    actions_prepared[idx]["minus"] = actions_prepared[idx]["mean"] - actions_prepared[idx]["std"]
    actions_prepared[idx]["plus"] = actions_prepared[idx]["mean"] + actions_prepared[idx]["std"]
# Actions Analysis Combi 2
from src.main.rl.utils.constants import scaling_factors_scenario_1, action_dimensions_german

fig, ax = plt.subplots(1, 5, constrained_layout=True)
fig.set_figheight(3)
fig.set_figwidth(20)
[ax[idx].plot(item["mean"], color=color_mapping["standard"]) for idx, item in enumerate(actions_prepared)]
# [ax[idx].fill_between([i for i in range(250)],item["plus"],item["minus"],color=color_mapping["standard"],alpha=0.3) for idx, item in enumerate(result_list)]
[
    ax[idx].fill_between([i for i in range(251)], item["min"], item["max"], color=color_mapping["standard"], alpha=0.3)
    for idx, item in enumerate(actions_prepared)
]
[ax[idx].set_xlabel("Zeitschritte") for idx in range(5)]
[ax[idx].set_ylabel("Value") for idx in range(5)]
[ax[idx].set_title(action_dimensions_german[idx]) for idx in range(5)]

# fig.savefig(
#    f"src/main/rl/evaluation/plot_results/phase3_actions_plot_with_min_max_scen1_w.png",
#    format="png",
#    dpi=300,
# )
plt.show()

In [ ]:
### Single Reward per timestep ###

In [ ]:
import pandas as pd
from src.main.rl.utils.parser import parse_scenario_name, parse_wrapper
from src.main.rl.utils.combined_parser import parse_information_from_path
from src.main.rl.evaluation.eval import get_single_reward

paths = [
    "src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption4Wrapper_None_RewardOption2Wrapper_TD3_training_04_06",
    "src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_SAC_training_04_06",
    "src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06",
    "src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption4Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_PPO_training_04_06",
    "src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06",
    "src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption3Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_PPO_training_04_06",
]
list_combinations = [pd.DataFrame() for i in range(len(paths))]
for idx, path in enumerate(paths):

    for number in range(1, 11):
        result_dict = {}
        full_path = path + f"_{str(number)}"
        path_to_overhand = full_path + "/best_model.zip"

        action_wrapper, automation_wrapper, obs_wrapper, reward_wrapper = parse_wrapper(full_path)
        scenario, alg, wrapper_maker = parse_information_from_path(full_path)
        reward = get_single_reward(scenario, path_to_overhand, alg, wrapper_maker, episode_length=250)
        intermediate_df = pd.DataFrame()
        intermediate_df[full_path] = reward
        list_combinations[idx] = list_combinations[idx].join(intermediate_df, how="outer")

    print(list_combinations[idx].eq(1).sum().sum())

for idx, item in enumerate(list_combinations):
    list_combinations[idx] = item.agg(["mean", "min", "max", "std"], axis=1)

In [ ]:
import matplotlib.pyplot as plt
from src.main.rl.utils.constants import color_mapping

fig, ax = plt.subplots(2, 3, constrained_layout=True)
fig.set_figheight(4)
fig.set_figwidth(10)
[
    ax[0][idx].plot(item["mean"], color=color_mapping["standard"])
    for idx, item in enumerate(list_combinations)
    if idx < 3
]
[
    ax[1][idx - 3].plot(item["mean"], color=color_mapping["standard"])
    for idx, item in enumerate(list_combinations)
    if idx >= 3
]
[
    ax[0][idx].fill_between(
        [i for i in range(249)], item["min"], item["max"], color=color_mapping["standard"], alpha=0.3
    )
    for idx, item in enumerate(list_combinations)
    if idx < 3
]
[
    ax[1][idx - 3].fill_between(
        [i for i in range(249)], item["min"], item["max"], color=color_mapping["standard"], alpha=0.3
    )
    for idx, item in enumerate(list_combinations)
    if idx >= 3
]
[ax[0][idx].set_xlabel("Zeitschritte") for idx in range(6) if idx < 3]
[ax[1][idx - 3].set_xlabel("Zeitschritte") for idx in range(6) if idx >= 3]
[ax[0][idx].set_ylabel("Value") for idx in range(6) if idx < 3]
[ax[1][idx - 3].set_ylabel("Value") for idx in range(6) if idx >= 3]
[ax[0][idx].set_title(f"Kombination {idx+1}") for idx in range(6) if idx < 3]
[ax[1][idx - 3].set_title(f"Kombination {idx+1}") for idx in range(6) if idx >= 3]
plt.show()
fig.savefig(
    f"src/main/rl/evaluation/plot_results/phase3_reward_per_step_per_combi.png",
    format="png",
    dpi=300,
)

In [ ]:
fig, ax = plt.subplots(1, 1, constrained_layout=True)
fig.set_figheight(4)
fig.set_figwidth(8)
colors = [
    color_mapping["blue"],
    color_mapping["red"],
    color_mapping["grey"],
    color_mapping["yellow"],
    color_mapping["brown"],
    color_mapping["turquoise"],
]
[ax.plot(item["mean"], color=colors[idx]) for idx, item in enumerate(list_combinations)]
ax.set_xlabel("Zeitschritte")
ax.set_xlabel("Reward")
ax.set_title(f"Durschnittlicher Reward je Zeitschritt pro Kombination")
plt.plot([], c=color_mapping["blue"], label="Kombination 1")
plt.plot([], c=color_mapping["red"], label="Kombination 2")
plt.plot([], c=color_mapping["grey"], label="Kombination 3")
plt.plot([], c=color_mapping["yellow"], label="Kombination 4")
plt.plot([], c=color_mapping["brown"], label="Kombination 5")
plt.plot([], c=color_mapping["turquoise"], label="Kombination 6")
plt.legend()
fig.savefig(
    f"src/main/rl/evaluation/plot_results/phase3_reward_per_step_per_combi.png",
    format="png",
    dpi=300,
)

In [ ]:
###### Statistics over the all combinations ######

In [ ]:
import pandas as pd
from scipy.stats import iqr
from src.main.rl.evaluation.phase3_evaluation import create_evaluation_df_phase3

paths = [
    "src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption4Wrapper_None_RewardOption2Wrapper_TD3_training_04_06",
    "src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_SAC_training_04_06",
    "src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06",
    "src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption4Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_PPO_training_04_06",
    "src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06",
    "src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_A2C_training_04_06",
]
path_to_save = "src/main/rl/evaluation/output/phase3_evaluation_results_new3.csv"
pd.options.display.max_colwidth = 500
df = pd.DataFrame()
try:
    df = pd.read_csv(path_to_save)
except:
    pass

if df.empty:
    df = create_evaluation_df_phase3(path_to_save, paths)
df

In [ ]:
# Statistics over the whole dataframe
df_total1 = df.drop(
    columns=["full_path", "combination", "scenario", "alg", "action_wrapper", "obs_wrapper", "automation_wrapper"]
).agg(["mean", "max", "min", "std"])
df_total2 = (
    df.drop(
        columns=["full_path", "combination", "scenario", "alg", "action_wrapper", "obs_wrapper", "automation_wrapper"]
    )
    .groupby(lambda x: True, dropna=True)
    .agg([iqr])
)
df_total2.columns = df_total2.columns.get_level_values(0)
df_total2["index"] = "iqr"
df_total = df_total1.reset_index().merge(df_total2, how="outer").set_index("index")

In [ ]:
# Statistics per combination
df_statistics_per_combination = (
    df.drop(columns=["full_path"])
    .groupby(["combination", "scenario", "alg", "action_wrapper", "obs_wrapper", "automation_wrapper"], dropna=False)
    .agg(["mean", "max", "min", "std", iqr])
)
# Necessary to set index as alphabetical is confusing for the thesis
df_statistics_per_combination["index"] = [0, 1, 3, 2, 5, 4]
df_statistics_per_combination = (
    df_statistics_per_combination.reset_index().set_index(["index", "combination"]).sort_index()
)
df_statistics_per_combination.columns = ["_".join(a) for a in df_statistics_per_combination.columns.to_flat_index()]
save_df_per_combination = df_statistics_per_combination.copy()
df_statistics_per_combination

In [ ]:
# Criticality Score per combination
print(df_statistics_per_combination[["criticality_score_mean"]].round(2))
print(
    df_statistics_per_combination[
        [
            "criticality_score_mean",
            "criticality_score_max",
            "criticality_score_min",
            "criticality_score_std",
            "criticality_score_iqr",
        ]
    ]
    .droplevel(1)
    .transpose()
    .round(2)
    .to_latex()
)